# FINAL MOVE

In [1]:
import os
# import nibabel as nib
# from nibabel.testing import data_path
import numpy as np
import cv2 
from matplotlib import pyplot as plt 
import sys
import shutil
from pathlib import Path


In [2]:
import pandas as pd

def createMetaCombinedString(meta_df):
    meta_image = meta_df.loc[:,"Image Data ID"]
    meta_subject = meta_df.loc[:,"Subject"]
    meta_visit = meta_df.loc[:,"Visit"]

    meta_combined= []
    i=0
    for i in range(len(meta_image)):
        # combined = str(meta_subject[i])+"-"+str(meta_series[i])+"-"+str(meta_image[i])
        combined = str(meta_subject[i])+"-"+str(meta_image[i]) #to check data availability based on subject and image id
        meta_combined.append(combined)
        i+=1

    return meta_combined

def exportCSV(meta_dict, title):
    temp_meta_df = pd.DataFrame(meta_dict) #convert dict to dataframe
    meta_csv_file = "./TempMeta/{}.csv".format(title)
    with open(meta_csv_file, mode='w') as f:
        temp_meta_df.to_csv(f)

    return temp_meta_df

In [ ]:
from pathlib import Path
# path ="./ADNI/"

def movePreprocessed(meta_df, path, seq, cond, tesla=3, divider="raw_"):
    # target_meta = "./metadata/"
    target_path = "./preprocessed/"
    # path = "./preprocessed_old/{}/{}/".format(seq, cond)

    path = path + "/{}/{}/".format(seq, cond)
    print(path)
    meta_nums = []
    result = list(Path(path).glob('**/wm*.nii'))
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number of files: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    # ctr = 0
    notsim = 0
    j=0
    target_dir = os.path.join(target_path, seq)
    target_dir = os.path.join(target_dir, cond)

    meta_combined = createMetaCombinedString(meta_df)
    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict.clear()

    meta_dict = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    
    notsim = 0
    for j in range(len(meta_combined)):

        flag = 0 #flag that there is matched data
        for f in result[:]:
            pathToFile = str(f)
            # print(pathToFile)
            fileName = pathToFile.split("\\")[5] #use "\\" if "/" does not work
            # print(fileName)
            fileNameNoExt = fileName.split(".nii")[0]

            part = fileNameNoExt.split('_MR')
            id_subject = part[0].split("ADNI_")[1]            
            part = part[1].split(divider)[1]
            id_series = part.split("_S")[1]
            id_series = id_series.split("_I")[0]
            id_image = fileNameNoExt.split("_I")[1]
        
            combinedIDs = id_subject+"-I"+id_image
            subdirName = id_subject+"-"+id_series+"-"+id_image
            
            
            if combinedIDs in meta_combined[j]: #Files already preprocessed             
                isExist = os.path.exists(target_dir)    

                if isExist == False:
                    os.mkdir(target_dir)
                
                shutil.copy(f,target_dir+"/{}-".format(j)+fileName)
                
                flag = 1

        if flag==1: #Files are preprocessed
            sim+=1      

        else: #Listing the not-yet-preprocessed files
            meta_dict["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
            meta_dict["Subject"].append(meta_df.loc[j,"Subject"])
            meta_dict["Group"].append(meta_df.loc[j,"Group"])
            meta_dict["Sex"].append(meta_df.loc[j,"Sex"])
            meta_dict["Age"].append(meta_df.loc[j,"Age"])
            meta_dict["Visit"].append(meta_df.loc[j,"Visit"])
            meta_dict["Modality"].append(meta_df.loc[j,"Modality"])
            meta_dict["Description"].append(meta_df.loc[j,"Description"])
            meta_dict["Type"].append(meta_df.loc[j,"Type"])
            meta_dict["Acq Date"].append(meta_df.loc[j,"Acq Date"])
            meta_dict["Format"].append( meta_df.loc[j,"Format"])   
            meta_nums.append(j)
            notsim +=1          
                
        j+=1
    # ctr+=1   
    # print(ctr)
    print("Total {}w-{} data is {} and not sim is {}".format(seq, cond, sim, notsim))
    return meta_dict, meta_nums

def freemove(source_path=None, target_path=None, seq = None, cond = None, tesla=3, file_format='**/*wm*.nii'): #move anything based on filename
    # target_meta = "./metadata/"
    # path = "./preprocessed_old/{}/{}/".format(seq, cond)
    target_dir = os.path.join(target_path, seq)
    target_dir = os.path.join(target_dir, cond)

    # Create target directory and parent directories if they don't exist
    os.makedirs(target_dir, exist_ok=True)

    path = source_path + "/{}/{}/".format(seq, cond)

    print(path)
    print(Path(path).exists())   
    meta_nums = []
    result = sorted(list(Path(path).glob(file_format))) #modify it to catch certain chars
    unique = set(result)

    print(f'----\n{seq}-{cond}\nOriginal number of files: {len(result)}\nUnique result: {len(unique)}')
  
    j =0
    for f in result[:]:
        pathToFile = str(f)
        # print(pathToFile)
        fileName = pathToFile.split("\\")[5] #use "\\" if "/" does not work
        # print(fileName)
        
        # if "wm" not in fileName and "y_" not in fileName and "mwp" not in fileName and "p0" not in fileName: #Files already preprocessed             
        if "ADNI" in fileName:
            print(fileName)
            isExist = os.path.exists(target_dir)    

            if isExist == False:
                os.mkdir(target_dir)
            
            shutil.copy(f,target_dir+"/{}-".format(j)+fileName)
            j=j+1

# FREE MOVE

In [65]:
freemove("./processed/", "./final/", "T2", "AD")
freemove("./processed", "./final/","T2", "MCI")
freemove("./processed/", "./final/","T2", "CN")

./processed//T2/AD/
True
----
T2-AD
Original number of files: 19
Unique result: 19
wmADNI_023_S_0139_MR_Axial_PD-T2_TSE__br_raw_20060728194739028_49_S17399_I19858.nii
wmADNI_023_S_0139_MR_Axial_PD-T2_TSE__br_raw_20080304124514156_49_S46596_I95268.nii
wmADNI_023_S_0916_MR_Axial_PD-T2_TSE__br_raw_20070424112956343_49_S30417_I50448.nii
wmADNI_023_S_0916_MR_Axial_PD-T2_TSE__br_raw_20070928145226219_49_S40716_I76278.nii
wmADNI_023_S_0916_MR_Axial_PD-T2_TSE__br_raw_20080930152607712_49_S56847_I118634.nii
wmADNI_023_S_1262_MR_Axial_PD-T2_TSE__br_raw_20070905093026173_96_S38816_I71708.nii
wmADNI_023_S_1262_MR_Axial_PD-T2_TSE__br_raw_20080402105251250_49_S47811_I100111.nii
wmADNI_023_S_1262_MR_Axial_PD-T2_TSE__br_raw_20090313141845859_49_S64535_I138900.nii
wmADNI_023_S_1289_MR_Axial_PD-T2_TSE__br_raw_20070813114426234_96_S37217_I67286.nii
wmADNI_033_S_0724_MR_Axial_PD_T2_FSE__br_raw_20080812160418472_2_S55319_I116026.nii
wmADNI_033_S_0733_MR_Axial_PD_T2_FSE__br_raw_20070814103742160_2_S37299_I6

In [66]:
freemove("./processed/", "./final/", "T1", "AD")
freemove("./processed", "./final/", "T1", "MCI")
freemove("./processed/", "./final/", "T1", "CN")

./processed//T1/AD/
True
----
T1-AD
Original number of files: 19
Unique result: 19
wmADNI_023_S_0139_MR_MPRAGE_br_raw_20060728171037231_1_S17389_I19846.nii
wmADNI_023_S_0139_MR_MPRAGE_Repeat__br_raw_20080304131137156_1_S46605_I95278.nii
wmADNI_023_S_0916_MR_MPRAGE_Repeat__br_raw_20070425122204375_1_S30559_I50801.nii
wmADNI_023_S_0916_MR_MPRAGE_REPEAT__br_raw_20070928154123360_1_S40737_I76308.nii
wmADNI_023_S_0916_MR_MPRAGE_br_raw_20080930154418859_1_S56855_I118643.nii
wmADNI_023_S_1262_MR_MPRAGE_Repeat__br_raw_20070905101106642_73_S38824_I71717.nii
wmADNI_023_S_1262_MR_MPRAGE_br_raw_20080402112037437_1_S47824_I100130.nii
wmADNI_023_S_1262_MR_MPRAGE_REPEAT__br_raw_20090313150930812_1_S64542_I138918.nii
wmADNI_023_S_1289_MR_MPRAGE_br_raw_20070813122635156_1_S37225_I67303.nii
wmADNI_033_S_0724_MR_MPRAGE_br_raw_20080812154746654_98_S55312_I116009.nii
wmADNI_033_S_0733_MR_MPRAGE_br_raw_20070814121736400_98_S37309_I67598.nii
wmADNI_033_S_0733_MR_MPRAGE_Repeat__br_raw_20080808152343320_138_S5

# Moving Already Preprocessed Files
------
Moving already prerpocessed files from the source (previous preprocessed folder) to the target folder and keep the list of the image that is not yet preprocessed

**Current Steps:**
1. Preprocessed files are taken from the folder `/preprocess_old` and saved to `/preprocessed` 
<br><br>-The preprocessed files are saved with this filename: `/preprocessed/{meta_id}_{filename}.nii.gz`
<br>-`{meta_id}` is the serial number of the file shown in the csv list
<br>-The idea of giving `{meta_id}` is to handle pairing of T1 and T2 data together


2. The list of not yet preprocessed files is saved as `TempMeta/To-Be-Preprocessed_{seq}w_{group}.csv`
-----
**Further steps:**
The list in Step #2 will be used to move the to-be-prerprocessed files to folder "/TempData" <br> This way the files can be transferred to the windows computer to be preprocessed

### Moving Old Preprocessed Files

In [20]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

In [ ]:
target_path = "./processed/"
path = "./preprocessed_old"

dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "T1", "AD")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "T1", "MCI")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "T1", "CN")
dict_2_ad, meta_num_2_ad = movePreprocessed(meta_df_2_ad, path,  "T2", "AD")
dict_2_mci, meta_num_2_mci = movePreprocessed(meta_df_2_mci, path, "T2", "MCI")
dict_2_cn, meta_num_2_cn = movePreprocessed(meta_df_2_cn, path,  "T2", "CN")

In [ ]:
print(meta_num_1_ad)
print(meta_num_1_mci)
print(meta_num_1_cn)
print(meta_num_2_ad)
print(meta_num_2_mci)
print(meta_num_2_cn)

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="To-Be-Preprocessed_{}w_{}".format("T1", "AD"))
tobe_df_1_mci = exportCSV(dict_1_mci, title="To-Be-Preprocessed_{}w_{}".format("T1", "MCI"))
tobe_df_1_cn = exportCSV(dict_1_cn, title="To-Be-Preprocessed_{}w_{}".format("T1", "CN"))
tobe_df_2_ad = exportCSV(dict_2_ad, title="To-Be-Preprocessed_{}w_{}".format("T2", "AD"))
tobe_df_2_mci = exportCSV(dict_2_mci, title="To-Be-Preprocessed_{}w_{}".format("T2", "MCI"))
tobe_df_2_cn = exportCSV(dict_2_cn, title="To-Be-Preprocessed_{}w_{}".format("T2", "CN"))


### Moving Additional Preprocessed Files

In [ ]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

path = "./preprocessed_addition/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci =  "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/Balanced_Meta_{}w_{}.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

In [ ]:
target_path = "./preprocessed/"
path = "./TempData"

dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "T1", "AD")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "T1", "MCI")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "T1", "CN")
dict_2_ad, meta_num_2_ad = movePreprocessed(meta_df_2_ad, path,  "T2", "AD")
dict_2_mci, meta_num_2_mci = movePreprocessed(meta_df_2_mci, path, "T2", "MCI")
dict_2_cn, meta_num_2_cn = movePreprocessed(meta_df_2_cn, path,  "T2", "CN")

# Moving Files to be Preprocessed

----
Moving files that are not yet preprocessed to the target folder, based on the list stored in metadata generated from the previous step. 

**Current Steps:**
1. The not yet preprocessed files re taken from the original downloaded folder `/3T/` and saved to `/TempData/` 
<br><br>-The files are saved with this filename: `/TempMeta/{filename}.nii.gz`

2. The data will be preprocessed in windows PC using MATLAB, therefore need to be put inside a decent sub-directory that contains `{subject id}-{series id}-{image id}`
-----
**Further steps:**
 After the data get preprocessed, we can go on with the third section that can be performed using the functions and algorithms in the first section

In [11]:
tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)
# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

20250213-111840


In [38]:
meta_df_1_ad

,Unnamed: 0,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,0,I55710,116_S_1083,AD,F,73,m06,MRI,MPRAGE Repeat,Original,5/25/2007,DCM
1,1,I57182,116_S_0392,AD,F,86,m12,MRI,MPRAGE Repeat,Original,6/15/2007,DCM
2,2,I126033,067_S_0812,AD,F,74,m24,MRI,MPRAGE Repeat,Original,11/10/2008,DCM
3,3,I115775,033_S_0733,AD,F,59,m24,MRI,MPRAGE Repeat,Original,8/07/2008,DCM
4,4,I67598,033_S_0733,AD,F,58,m12,MRI,MPRAGE,Original,8/10/2007,DCM
5,5,I67303,023_S_1289,AD,F,78,m06,MRI,MPRAGE,Original,8/10/2007,DCM
6,6,I100130,023_S_1262,AD,F,74,m12,MRI,MPRAGE,Original,4/02/2008,DCM
7,7,I71717,023_S_1262,AD,F,73,m06,MRI,MPRAGE Repeat,Original,9/04/2007,DCM
8,8,I138918,023_S_1262,AD,F,75,m24,MRI,MPRAGE_REPEAT,Original,3/13/2009,DCM
9,9,I19846,023_S_0139,AD,F,66,m06,MRI,MPRAGE,Original,7/28/2006,DCM


In [21]:
def move2preprocess(meta_df, seq, cond, tesla=3, divider="raw_"):
    # nii_path = "./{}T/{}/{}/".format(tesla, seq, cond)
    nii_path = "./DICOM/{}/{}/".format(seq, cond)
    print(nii_path)

    target_path = "./TempData/"

    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    for f in result[:]:
        pathToFile = str(f)
        # print(pathToFile)
        fileName = pathToFile.split("\\")[8] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        # print(fileNameNoExt)
        part = fileNameNoExt.split('_MR')

        id_subject = part[0].split("ADNI_")[1]

        # part = part[1].split(divider)[1]

        if "raw_" in part[1]:
            part = part[1].split("raw_")[1]
        else:
            part = part[1].split("br_")[1]

        id_series = part.split("_S")[1]
        id_series = id_series.split("_I")[0]
        id_image = fileNameNoExt.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        
        target_dir = os.path.join(target_path, seq)
        target_dir = os.path.join(target_dir, cond)
        target_dir = os.path.join(target_dir, subdirName)
        isExist = os.path.exists(target_dir)    

        j = 0
        for j in range(len(meta_combined)):
            if meta_combined[j] in combinedIDs:

                if isExist == False:
                    os.mkdir(target_dir)
                # print(target_dir)
                print("File {} copied to target folder".format(fileName))
                shutil.copy(f,target_dir+"/"+fileName)

                sim+=1
            j+=1
        ctr+=1    
        # print("File {}".format(ctr))
    print("Total {}w - {} data is {}".format(seq, cond, sim))

In [ ]:
move2preprocess(meta_df_1_ad, "T1", "AD")
move2preprocess(meta_df_1_mci, "T1", "MCI")
move2preprocess(meta_df_1_cn, "T1", "CN")
move2preprocess(meta_df_2_ad, "T2", "AD")
move2preprocess(meta_df_2_mci, "T2", "MCI")
move2preprocess(meta_df_2_cn, "T2", "CN")

./DICOM/T2/MCI/
---------
T2w-MCI
Original number: 869
Unique result:869
Total T2w - MCI data is 0


# Data Separation For Robustness Evaluation

The idea is to move the new datasets (hold-out data) for robustness evaluation to be preprocessed.
The hold-out datasets include 3T and 1.5T MRI. Hold-out data has never been used as the training data (the initial 5-fold data) --refer to `Balanced_Meta_{seq}w_{group}` for comparison.

In [ ]:

from pathlib import Path
# path = "C:/Users/Brain-LPDP/Documents/DEWINDA PMDSU/Publication/2023 - Research Visit KHU/Experiment/ADNI1 All Data/ADNI"
# path ="./ADNI/"

def find2separate(meta_df, ref_df, seq, tesla=3):
    nii_path = "./DataOri/{}T/{}/".format(tesla, seq)

    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    unique = set(result)
    print("---------\n{}w\nOriginal number: {}\nUnique result:{}".format(seq, len(result), len(unique)))

    sim = 0
    notsim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)
    ref_combined = createMetaCombinedString(ref_df)

    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_ad.clear()
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci.clear()
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn.clear()
    
    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    
    #Prepared array to be exported to CSV (separated per class)
    meta_nums_ad = []
    meta_nums_mci = []
    meta_nums_cn = []

    #Loop to check if there is already used data
    for j in range(len(meta_combined)):
        flag = 0 #flag that there is matched data

        for k in range(len(ref_combined)):

            # if ref_combined[k] in meta_combined[j]: #Similarities based on subject and imageid
            if meta_df.loc[j,"Subject"] in ref_df.loc[k,"Subject"]:  #similarities based on subject id only
                print("{} and {}".format(meta_combined[j], ref_combined[k]))
                flag = 1

        if flag==1: #Files are already used
            print("flag")
            sim = sim + 1    

        else: #Listing the not-yet-used files to be preprocessed
            if "AD" in meta_df.loc[j,"Group"]:
                meta_dict_ad["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_ad["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_ad["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_ad["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_ad["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_ad["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_ad["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_ad["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_ad["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_ad["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_ad["Format"].append( meta_df.loc[j,"Format"])

            elif "MCI" in meta_df.loc[j,"Group"]:
                meta_dict_mci["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_mci["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_mci["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_mci["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_mci["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_mci["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_mci["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_mci["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_mci["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_mci["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_mci["Format"].append( meta_df.loc[j,"Format"])
            else:
                meta_dict_cn["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                meta_dict_cn["Subject"].append(meta_df.loc[j,"Subject"])
                meta_dict_cn["Group"].append(meta_df.loc[j,"Group"])
                meta_dict_cn["Sex"].append(meta_df.loc[j,"Sex"])
                meta_dict_cn["Age"].append(meta_df.loc[j,"Age"])
                meta_dict_cn["Visit"].append(meta_df.loc[j,"Visit"])
                meta_dict_cn["Modality"].append(meta_df.loc[j,"Modality"])
                meta_dict_cn["Description"].append(meta_df.loc[j,"Description"])
                meta_dict_cn["Type"].append(meta_df.loc[j,"Type"])
                meta_dict_cn["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                meta_dict_cn["Format"].append( meta_df.loc[j,"Format"]) 

            notsim +=1          
        j+=1
        
    print("Total of used {}w data is {}".format(seq, sim))
    return meta_dict_ad, meta_dict_mci, meta_dict_cn

def move2separate(meta_df, seq, tesla=3, ONLY_BASELINE=False, divider="Br_"):
    nii_path = "./DataOri/{}T/{}/".format(tesla, seq)
    target_path = "./DataSep/"

    tesla = str(tesla) + "T"
    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    unique = set(result)
    print("---------\n{}w\nOriginal number: {}\nUnique result:{}".format(seq, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    #Initialize dict to save metadata
    meta_dict_ad = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_mci = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}
    meta_dict_cn = {"Image Data ID" : [], "Subject":[], "Group":[],"Sex":[],"Age":[],"Visit":[],"Modality":[],"Description":[],"Type":[],"Acq Date":[],"Format":[]}

    for f in result[:]:
        pathToFile = str(f)
        #print(pathToFile)
        fileName = pathToFile.split("\\")[7] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        #print(fileNameNoExt)
        part = fileNameNoExt.split('_MR')
        id_subject = part[0].split("ADNI_")[1]
        part = part[1].split(divider)[1] #raw if T2, Br if T1?
        id_series = part.split("_S")[1]
        id_series = id_series.split("_I")[0]
        id_image = fileNameNoExt.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        
        j = 0
        for j in range(len(meta_combined)):
            flag = 0
            if meta_combined[j] in combinedIDs:
                target_dir = os.path.join(target_path, tesla)
                target_dir = os.path.join(target_dir, meta_df.loc[j,"Group"])
                target_dir = os.path.join(target_dir, subdirName)
                
                isExist = os.path.exists(target_dir)
                
                if ONLY_BASELINE: #adjust the conditions
                    # if "AD" in meta_df.loc[j, "Group"]:
                    if "sc" in meta_df.loc[j, "Visit"] and "MPR-R" not in meta_df.loc[j, "Description"]: #Take only screening for 15T MRI
                    # if "bl" in meta_df.loc[j, "Visit"] or "m06" in meta_df.loc[j, "Visit"] or "m12" in meta_df.loc[j, "Visit"]: #Take only baseline from MCI and CN
                        if isExist == False:
                            os.mkdir(target_dir)
                        # print(target_dir)
                        print("File {} copied to target folder".format(fileName))
                        shutil.copy(f,target_dir+"/"+fileName)
                        flag=1
                else:
                    if isExist == False:
                        os.mkdir(target_dir)
                    # print(target_dir)
                    print("File {} copied to target folder".format(fileName))
                    shutil.copy(f,target_dir+"/"+fileName)
                    flag=1    
                sim+=1
            
            if flag==1: #Files are already used
                if "AD" in meta_df.loc[j,"Group"]:
                    meta_dict_ad["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_ad["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_ad["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_ad["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_ad["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_ad["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_ad["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_ad["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_ad["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_ad["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_ad["Format"].append( meta_df.loc[j,"Format"])   
                    
                elif "MCI" in meta_df.loc[j,"Group"]:
                    meta_dict_mci["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_mci["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_mci["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_mci["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_mci["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_mci["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_mci["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_mci["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_mci["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_mci["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_mci["Format"].append( meta_df.loc[j,"Format"])   
                    
                else:
                    meta_dict_cn["Image Data ID"].append(meta_df.loc[j,"Image Data ID"])
                    meta_dict_cn["Subject"].append(meta_df.loc[j,"Subject"])
                    meta_dict_cn["Group"].append(meta_df.loc[j,"Group"])
                    meta_dict_cn["Sex"].append(meta_df.loc[j,"Sex"])
                    meta_dict_cn["Age"].append(meta_df.loc[j,"Age"])
                    meta_dict_cn["Visit"].append(meta_df.loc[j,"Visit"])
                    meta_dict_cn["Modality"].append(meta_df.loc[j,"Modality"])
                    meta_dict_cn["Description"].append(meta_df.loc[j,"Description"])
                    meta_dict_cn["Type"].append(meta_df.loc[j,"Type"])
                    meta_dict_cn["Acq Date"].append(meta_df.loc[j,"Acq Date"])
                    meta_dict_cn["Format"].append( meta_df.loc[j,"Format"])   
                
            j+=1
            
    print("Total {}w data moved is {}".format(seq, sim))
    return meta_dict_ad, meta_dict_mci, meta_dict_cn

### Check Already Used Data

In [ ]:
Tesla = 3

meta_path_cn_t1w = "./TempMeta/Unique_Matched_Nolimit_T1w_CN_ADNI_3T_11_22_2023.csv"
meta_path_ad_t1w = "./TempMeta/Unique_Matched_Nolimit_T1w_AD_ADNI_3T_11_22_2023.csv"
meta_path_mci_t1w = "./TempMeta/Unique_Matched_Nolimit_T1w_MCI_ADNI_3T_11_22_2023.csv"
# Reference metadata (Already used for training)
meta_ad_t1w = "./Metadata/Balanced_Meta_T1w_AD.csv"
meta_mci_t1w = "./Metadata/Balanced_Meta_T1w_MCI.csv"
meta_cn_t1w = "./Metadata/Balanced_Meta_T1w_CN.csv"

meta_path_cn_t2w = "./TempMeta/Unique_Matched_Nolimit_T2w_CN_ADNI_3T_11_22_2023.csv"
meta_path_ad_t2w = "./TempMeta/Unique_Matched_Nolimit_T2w_AD_ADNI_3T_11_22_2023.csv"
meta_path_mci_t2w = "./TempMeta/Unique_Matched_Nolimit_T2w_MCI_ADNI_3T_11_22_2023.csv"
# Reference metadata (Already used for training)
meta_ad_t2w = "./Metadata/Balanced_Meta_T2w_AD.csv"
meta_mci_t2w = "./Metadata/Balanced_Meta_T2w_MCI.csv"
meta_cn_t2w = "./Metadata/Balanced_Meta_T2w_CN.csv"


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))

# Read CSV files into DataFrames
meta_df_ad_t1w = pd.read_csv(meta_path_ad_t1w, delimiter=',')
meta_df_cn_t1w = pd.read_csv(meta_path_cn_t1w, delimiter=',')
meta_df_mci_t1w = pd.read_csv(meta_path_mci_t1w, delimiter=',')
meta_df_ad_t2w = pd.read_csv(meta_path_ad_t2w, delimiter=',')
meta_df_cn_t2w = pd.read_csv(meta_path_cn_t2w, delimiter=',')
meta_df_mci_t2w = pd.read_csv(meta_path_mci_t2w, delimiter=',')
meta_df_ad_t1w_ref = pd.read_csv(meta_ad_t1w, delimiter=',')
meta_df_mci_t1w_ref = pd.read_csv(meta_mci_t1w, delimiter=',')
meta_df_cn_t1w_ref = pd.read_csv(meta_cn_t1w, delimiter=',')
meta_df_ad_t2w_ref = pd.read_csv(meta_ad_t2w, delimiter=',')
meta_df_cn_t2w_ref = pd.read_csv(meta_cn_t2w, delimiter=',')
meta_df_mci_t2w_ref = pd.read_csv(meta_mci_t2w, delimiter=',')

20231129-224822


In [ ]:
# T1w combined DataFrames
combined_df_t1w_ref = pd.concat([meta_df_ad_t1w_ref, meta_df_mci_t1w_ref, meta_df_cn_t1w_ref], ignore_index=True)
combined_df_t1w = pd.concat([meta_df_ad_t1w, meta_df_mci_t1w, meta_df_cn_t1w], ignore_index=True)

# T2w combined DataFrames
combined_df_t2w_ref = pd.concat([meta_df_ad_t2w_ref, meta_df_mci_t2w_ref, meta_df_cn_t2w_ref], ignore_index=True)
combined_df_t2w = pd.concat([meta_df_ad_t2w, meta_df_mci_t2w, meta_df_cn_t2w], ignore_index=True)


In [ ]:
dict_1_ad, dict_1_mci, dict_1_cn = find2separate(combined_df_t1w,combined_df_t1w_ref, "T1", Tesla)
dict_2_ad, dict_2_mci, dict_2_cn = find2separate(combined_df_t2w,combined_df_t2w_ref, "T2", Tesla)

---------
T1w
Original number: 0
Unique result:0
136_S_0299-I29120 and 136_S_0299-I29120
136_S_0299-I29120 and 136_S_0299-I15181
flag
136_S_0299-I105390 and 136_S_0299-I29120
136_S_0299-I105390 and 136_S_0299-I15181
flag
136_S_0299-I15181 and 136_S_0299-I29120
136_S_0299-I15181 and 136_S_0299-I15181
flag
136_S_0194-I14155 and 136_S_0194-I26619
136_S_0194-I14155 and 136_S_0194-I47814
136_S_0194-I14155 and 136_S_0194-I14154
flag
136_S_0194-I47815 and 136_S_0194-I26619
136_S_0194-I47815 and 136_S_0194-I47814
136_S_0194-I47815 and 136_S_0194-I14154
flag
136_S_0194-I26619 and 136_S_0194-I26619
136_S_0194-I26619 and 136_S_0194-I47814
136_S_0194-I26619 and 136_S_0194-I14154
flag
131_S_0457-I110025 and 131_S_0457-I16523
131_S_0457-I110025 and 131_S_0457-I30779
131_S_0457-I110025 and 131_S_0457-I60917
flag
131_S_0457-I60916 and 131_S_0457-I16523
131_S_0457-I60916 and 131_S_0457-I30779
131_S_0457-I60916 and 131_S_0457-I60917
flag
131_S_0457-I16523 and 131_S_0457-I16523
131_S_0457-I16523 and 131_

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="HoldOut_{}w_{}_{}T".format("T1", "AD", Tesla))
tobe_df_1_mci = exportCSV(dict_1_mci, title="HoldOut_{}w_{}_{}T".format("T1", "MCI", Tesla))
tobe_df_1_cn = exportCSV(dict_1_cn, title="HoldOut_{}w_{}_{}T".format("T1", "CN", Tesla))

tobe_df_2_ad = exportCSV(dict_2_ad, title="HoldOut_{}w_{}_{}T".format("T2", "AD", Tesla))
tobe_df_2_mci = exportCSV(dict_2_mci, title="HoldOut_{}w_{}_{}T".format("T2", "MCI", Tesla))
tobe_df_2_cn = exportCSV(dict_2_cn, title="HoldOut_{}w_{}_{}T".format("T2", "CN", Tesla))

In [ ]:
print(tobe_df_1_ad.shape)
print(tobe_df_1_mci.shape)
print(tobe_df_1_cn.shape)
print(tobe_df_2_ad.shape)
print(tobe_df_2_mci.shape)
print(tobe_df_2_cn.shape)

(19, 11)
(696, 11)
(49, 11)
(19, 11)
(696, 11)
(49, 11)


### Move New Data To Be Preprocessed

In [ ]:
Tesla = 15

# meta_path = "./Metadata/ADNI1_Baseline_3T_6_08_2023.csv" #Meta path for the target data
# meta_path = "./Metadata/ADNI1_Complete_3Yr_3T_6_08_2023.csv"
meta_path = "./Metadata/ADNI1_Complete_1Yr_15T_6_08_2023.csv"

# Reference metadata (Already used for training)
meta_ad = "./TempMeta/HoldOut_T1w_AD_{}T.csv".format(Tesla)
meta_mci = "./TempMeta/HoldOut_T1w_MCI_{}T.csv".format(Tesla)
meta_cn = "./TempMeta/HoldOut_T1w_CN_{}T.csv".format(Tesla)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df = pd.read_csv(meta_path, delimiter=',')
meta_df_ad = pd.read_csv(meta_ad, delimiter=',')
meta_df_mci = pd.read_csv(meta_mci, delimiter=',')
meta_df_cn = pd.read_csv(meta_cn, delimiter=',')

In [ ]:
combined_df = pd.concat([meta_df_ad, meta_df_mci, meta_df_cn], ignore_index=True)

In [ ]:
dict_1_ad, dict_1_mci, dict_1_cn = move2separate(combined_df, "T1", Tesla, ONLY_BASELINE=True)

In [ ]:
tobe_df_1_ad = exportCSV(dict_1_ad, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "AD", Tesla))
tobe_df_1_mci = exportCSV(dict_1_mci, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "MCI", Tesla))
tobe_df_1_cn = exportCSV(dict_1_cn, title="HoldOut_Cleaned_{}w_{}_{}T".format("T1", "CN", Tesla))

### Move Preprocessed Hold-out Data

In [ ]:
# tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)

# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
Tesla = 3
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/HoldOut_Balanced_{}w_{}_{}T.csv".format(seq, cond, Tesla)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')

In [ ]:
target_path = "./preprocessed/"
path = "./TempData"

dict_1_ad, meta_num_1_ad = movePreprocessed(meta_df_1_ad, path, "15T", "AD", divider="Br_")
dict_1_mci, meta_num_1_mci = movePreprocessed(meta_df_1_mci, path, "15T", "MCI", divider="Br_")
dict_1_cn, meta_num_1_cn = movePreprocessed(meta_df_1_cn, path, "15T", "CN", divider="Br_")

# Moving DICOM Files to be Converted to Niftii

This is used when you download DICOM files instead of Niftii from ADNI dataset. Recently (since around November 2023) ADNI no longer provides the files to be downloaded in DICOM format, especially for those that we searched ourselves (like through advanced search). Therefore, there is no other way but to use MRICron for dicom2nii converter (the application is provided ADNI as well).
However, when using this application for me, there was a problem with naming the output niftii files. I intend to name it the same name with their format (I use %b which is for naming as the base name), but they seem to be lacking of image ID and series ID. 

For example, this should be our niftii filename: `ADNI_002_S_0816_MR_MPRAGE_REPEAT__br_raw_20070329110738780_1_S29069_I47405.nii`. However, this is what I got from their converter app: `ADNI_002_S_0782_MR_Axial_PD_T2_FSE__br_raw_20060814235122397_2_S1_I20521_e2a.nii`.

This would be a bummer to create your own dicom to niftii converter and there might be some rules that you have to follow, so this is not practical at all to create your own python code. Therefore, my solution is to follow these steps:
- Move the files to a folder that has sub-folders with the series and image ID as the name of the sub-folders, example “S29096_I41124” (will be handled by function `move2convert`).
- Convert the dicom files after previous step is done
    - The naming for output niftii file is “%b_%f” in the MRIcron application for dicom2nii 
    - Converted niftii data will be put in one folder
- Move each converted niftii data inside a sub-folder that has a name id_subject**+**"-"**+**id_series**+**"-"**+**id_image” (i.e. 002_S_0001-S29096_I41124). This one will be handled by function `moveConverted`.

In [3]:
tesla = 3 #1.5 or 3
# downloadDate = "12_05_2022"
# path = "./{}T/{}/{}/".format(tesla, seq, cond)
# path = "./preprocessed_old/{}/{}/".format(seq, cond)

seq = "T1"
cond= "AD" #condition is AD or CN or MCI
meta_path_1_ad = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_1_mci = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_1_cn = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
seq = "T2"
cond= "AD" #condition is AD or CN or MCI
meta_path_2_ad = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "MCI" #condition is AD or CN or MCI
meta_path_2_mci = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
cond= "CN" #condition is AD or CN or MCI
meta_path_2_cn = "./TempMeta/HoldOut_{}w_{}_3T.csv".format(seq, cond)
# path = "./data/{}/{}/".format(seq, cond)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
print(datetime.now().strftime('%Y%m%d-%H%M%S'))
# meta_path = "./Metadata/metadata_final.csv"
meta_df_1_ad = pd.read_csv(meta_path_1_ad, delimiter=',')
meta_df_1_mci = pd.read_csv(meta_path_1_mci, delimiter=',')
meta_df_1_cn = pd.read_csv(meta_path_1_cn, delimiter=',')
meta_df_2_ad = pd.read_csv(meta_path_2_ad, delimiter=',')
meta_df_2_mci = pd.read_csv(meta_path_2_mci, delimiter=',')
meta_df_2_cn = pd.read_csv(meta_path_2_cn, delimiter=',')

20250218-121846


In [11]:
meta_df_2_mci

,Unnamed: 0,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format
0,0,I44493,941_S_1363,MCI,F,70,sc,MRI,Axial PD-T2 TSE,Original,3/12/2007,DCM
1,1,I46856,141_S_1378,MCI,F,62,sc,MRI,Axial PD-T2 TSE,Original,3/26/2007,DCM
2,2,I38114,141_S_1255,MCI,F,77,sc,MRI,Axial PD-T2 TSE,Original,2/03/2007,DCM
3,3,I37660,141_S_1245,MCI,F,71,sc,MRI,Axial PD-T2 TSE,Original,1/29/2007,DCM
4,4,I41220,141_S_1231,MCI,F,84,sc,MRI,Axial PD-T2 TSE,Original,2/18/2007,DCM
...,...,...,...,...,...,...,...,...,...,...,...,...
691,691,I11301,005_S_0222,MCI,M,86,sc,MRI,Axial PD/T2 FSE,Original,2/21/2006,DCM
692,692,I57549,002_S_1155,MCI,M,59,m06,MRI,Axial PD/T2 FSE,Original,6/21/2007,DCM
693,693,I33395,002_S_1155,MCI,M,58,sc,MRI,Axial PD/T2 FSE,Original,12/14/2006,DCM
694,694,I20521,002_S_0782,MCI,M,82,sc,MRI,Axial PD/T2 FSE,Original,8/14/2006,DCM


In [22]:
def move2convert(meta_df, seq, cond, tesla=3, divider="raw_"):
    dicom_path = "./DICOM/{}/{}/".format(seq, cond)
    print(dicom_path)
    target_path = "./2convert/"
    result = list(Path(dicom_path).glob('**/*.dcm'))
    unique = set(result)

    print("---------\n{}w-{}\nOriginal number: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    for f in result[:]:
        pathToFile = str(f)
        # print(pathToFile)
        fileName = pathToFile.split("\\")[8] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        # print(fileName)
        fileNameNoExt = fileName.split(".dcm")[0]
        
        part = fileNameNoExt.split('_MR')
        # id_subject = part[0].split("ADNI_")[1]
        if "ADNI_" in part[0]:
            id_subject = part[0].split("ADNI_")[1]
           
        else:
            id_subject = part[0]
            id_subject = id_subject[1:]
        # print(id_subject)
        # raw = part[1].split(divider)[1]
        if "raw_" in part[1]:
            raw = part[1].split(divider)[1]
        else:
            raw = part[1].split("br_")[1]

        if "_S" in raw:
            id_series = raw.split("_S")[1]
            id_series = id_series.split("_I")[0]
            id_image = fileNameNoExt.split("_I")[1]
        
            combinedIDs = id_subject+"-I"+id_image
            subdirName = "_S"+id_series+"_I"+id_image
            # subdirName = id_series+"_I"+id_image
            
            # print(combinedIDs)
            target_dir = os.path.join(target_path, seq)
            target_dir = os.path.join(target_dir, cond)
            target_dir = os.path.join(target_dir, subdirName)
            isExist = os.path.exists(target_dir)    

            j = 0
            for j in range(len(meta_combined)):
                # print(meta_combined[j])
                if meta_combined[j] in combinedIDs:
                    if isExist == False:
                        os.mkdir(target_dir)
                    # print(target_dir)
                    print("File {} copied to target folder".format(fileName))
                    shutil.copy(f,target_dir+"/"+fileName)

                    sim+=1
                j+=1
            ctr+=1    
        else:
            continue
        # print("File {}".format(ctr))
    print("Total {}w - {} data is {}".format(seq, cond, sim))

In [ ]:
move2convert(meta_df_1_ad, "T1", "AD")
move2convert(meta_df_1_mci, "T1", "MCI")
move2convert(meta_df_1_cn, "T1", "CN")
move2convert(meta_df_2_ad, "T2", "AD")
move2convert(meta_df_2_mci, "T2", "MCI")
move2convert(meta_df_2_cn, "T2", "CN")

./DICOM/T2/MCI/
---------
T2w-MCI
Original number: 104753
Unique result:104753
Total T2w - MCI data is 0


In [13]:
move2convert(meta_df_2_cn, "T2", "CN")

./DICOM/T2/CN/
---------
T2w-CN
Original number: 0
Unique result:0


UnboundLocalError: local variable 'j' referenced before assignment

# Move Converted

In [9]:
def moveConverted(meta_df, seq, cond, tesla=3, divider="br_"):
    # nii_path = "./{}T/{}/{}/".format(tesla, seq, cond)
    
    nii_path = "./Converted/{}/{}/".format(seq, cond)
    print(nii_path)

    target_path = "./TempData/"

    result = sorted(list(Path(nii_path).glob('**/*.nii')))
    print(result)
    unique = set(result)
    print("---------\n{}w-{}\nOriginal number: {}\nUnique result:{}".format(seq, cond, len(result), len(unique)))

    sim = 0
    ctr = 0
    meta_combined = createMetaCombinedString(meta_df)

    for f in result[:]:
        pathToFile = str(f)
        print(f)
        fileName = pathToFile.split("\\")[3] #use "\\" if "/" does not work 7 if from folder 3T/seq/group
        fileNameNoExt = fileName.split(".nii")[0]
        
        part = fileNameNoExt.split('_MR')
        id_subject = part[0].split("ADNI_")[1]
        # print(id_subject)
        part = part[1].split(divider)[1]
        print(part)
        series = part.split("__S")[1]
        id_series = series.split("_I")[0]
        id_image = series.split("_I")[1]
    
        combinedIDs = id_subject+"-I"+id_image
        subdirName = id_subject+"-"+id_series+"-"+id_image
        print(combinedIDs)
        
        target_dir = os.path.join(target_path, seq)
        target_dir = os.path.join(target_dir, cond)
        target_dir = os.path.join(target_dir, subdirName)
        isExist = os.path.exists(target_dir)    

        j = 0
        for j in range(len(meta_combined)):
            if meta_combined[j] in combinedIDs:

                if isExist == False:
                    os.mkdir(target_dir)
                # print(target_dir)
                # print("File {} copied to target folder".format(fileName))
                shutil.copy(f,target_dir+"/"+fileName)

                sim+=1
            j+=1
        ctr+=1    
        # print("File {}".format(ctr))
    print("Total {}w - {} data is {}".format(seq, cond, sim))

In [11]:
moveConverted(meta_df_1_mci, "T1", "MCI")
# moveConverted(meta_df_2_cn, "T2", "CN")

./Converted/T1/MCI/
[WindowsPath('Converted/T1/MCI/ADNI_002_S_0782_MR_MP-RAGE__br_raw_20060814233807496_1_S17835_I20__S17835_I20519.nii'), WindowsPath('Converted/T1/MCI/ADNI_002_S_0782_MR_MP-RAGE_REPEAT_br_raw_20070412010432956_1_S295__S29565_I48752.nii'), WindowsPath('Converted/T1/MCI/ADNI_002_S_1155_MR_MP-RAGE__br_raw_20061215203537627_1_S24144_I33__S24144_I33393.nii'), WindowsPath('Converted/T1/MCI/ADNI_002_S_1155_MR_MP-RAGE_REPEAT_br_raw_20070621143950664_1_S339__S33920_I57547.nii'), WindowsPath('Converted/T1/MCI/ADNI_005_S_0222_MR_MP-RAGE__br_raw_20060228162849855_1_S11753_I11__S11753_I11299.nii'), WindowsPath('Converted/T1/MCI/ADNI_005_S_0222_MR_MP-RAGE__br_raw_20061003151636200_1_S19685_I25__S19685_I25669.nii'), WindowsPath('Converted/T1/MCI/ADNI_005_S_0222_MR_MP-RAGE_REPEAT_br_raw_20070417133647958_1_S298__S29894_I49438.nii'), WindowsPath('Converted/T1/MCI/ADNI_005_S_0546_MR_MP-RAGE__br_raw_20060615120703023_1_S15567_I17__S15567_I17056.nii'), WindowsPath('Converted/T1/MCI/ADNI_